* 색인 & 가중치 => 학습
1. 수집한 문서 목록 가져오기
2. 각 문서에서 색인어 목록 추출 전처리 -> 어절 -> 형태소+명사+바이그램
3. 가중치 계산(TF-IDF) -> TDM -> TWM(with DocumentVectorLength)

* 질의 -> 색인과정의 2 to 3
4. 질의에서 색인어 추출
5. 가중치 계산
6. 유사도 계산(코싸인)
7. 유사도 순 정렬(거리-오름차순, 각도-내림차순)
8. 검색 결과 출력

In [1]:
def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

# txt 컨텐츠를 편하게 읽어오기 위한 함수
def getContent(file): 
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

In [2]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [3]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [4]:
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"를 2019 및 금지|네이버 채널에서|저작권자 주 블로터앤미디어 저작권자를 명기하고 내용을 변경하지 않으며 비상업적으로 이용하는 조건아래 가능합니다|"
                          r"디지털 마케팅의 미래 아이포럼|무단전재|오류를 우회하기 위한 함수 추가|재배포|기자|아이뉴스24|구독해주세요|"
                          r"고수들의 재테크 비법|이건희칼럼|성공을 꿈꾸는 사람들의 경제 뉴스|머니 및 금지|및 금지|"
                          r"네이버 모바일에서|네이버에서 헤럴드경제|채널 구독하기|헤럴드 리얼라이프 헤럴드경제 사이트 바로가기 헤럴드경제 무단 전재 및 금지|"
                          r"전자신문|바로가기|전자신문인터넷|인터넷 및 금지|중앙일보|친구추가|구독 1위|"
                          r"한국경제|재배포|기사제보|보도자료|한경닷컴|네이버에서|자세히보기|자세히 보기|모바일한경|구독신청|"
                          r"머니투데이|돈이 보이는 리얼타임 뉴스|연예뉴스|파이낸셜뉴스|무단 전재 금지") # 높은 가중치를 갖는 단어를 하드코딩 데이터로 제거
    patternList["ElimRecWord"] = pattern7
    
    return patternList

In [5]:
from os import listdir

FileList = getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")
NewsContent = list() 

for i in range(len(FileList)):
    NewsContent.append(getContent(FileList[i])) # len(NewsContent) = 180


In [6]:
patternList = getPatternList()
for _ in ["Korean", "Whitespace", "Punctuation", "ElimLongEng", "Email", "Domain", "ElimRecWord"]: 
    for i in range(len(FileList)):
        NewsContent[i] = patternList[_].sub(" ", NewsContent[i]) 
print(NewsContent[0])


   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과 유엔환경계획 에 국제 과학자 호소문 을 제출하기는 했다 당시 호소문에는 전세계 저명한 학술

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()
# len(sent_tokenize(NewsContent[0])) # 구두점이 사라져서 토큰화 못함.

dictTerm = list()
dictPos = list()
dictNoun = list()
dictNgram = list()
th=1

for sentence in sent_tokenize(NewsContent[0]):
    for token in word_tokenize(sentence):
        if len(token) > th:
            dictTerm.append(token)
            # 아래서부터는 list이기때문에 extend를 사용해야 함. 
            dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
            dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
            dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
            
# 빠른 속도 및 중복 데이터 처리를 위해 set사용
dictTerm = list(set(dictTerm))
dictPos = list(set(dictPos))
dictNoun = list(set(dictNoun))
dictNgram = list(set(dictNgram))

In [8]:
len(dictTerm), len(dictPos), len(dictNoun), len(dictNgram)


(276, 214, 181, 484)

In [9]:

len(list(set(dictTerm + dictPos + dictNoun + dictNgram)))


754

In [10]:
collection = []
# print(type(FileList[0]))

for i in range(len(NewsContent)):
    collection.append((str(FileList[i]), NewsContent[i]))
print(type(collection[10][1]))

<class 'str'>


In [11]:
# in-memory (Hash Key 값)
# 전체 색인어 목록(Dictionary)
# {단어1:포스팅위치, 단어2:포스팅위치, ...}
globalLexicon = dict()
queryPosting = dict()
# 전체 문서 목록(Dictionary)
# [0:문서1, 1:문서2, ...]
globalDocument = list()

# disk
# 사전에 있는 색인어 중, 어느 문서에서, 몇 번 나타났는지
# [(단어 idx, 문서 idx, 빈도, 다음주소), ...]
# [0:Tuple(lexiconIdx, documentIdx, freq, 다음포스팅위치-fptr)]
# 메모리 X, File OK
globalPosting = list()


In [12]:
for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # Local 작업
    for term in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if term not in localPosting.keys():
            localPosting[term] = 1
        # 있으면, 빈도 증가
        else:
            localPosting[term] += 1
     
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in queryPosting.keys(): 
            lexiconIdx = len(queryPosting)
            postingIdx = len(globalPosting) # fseek
            postingData = (lexiconIdx, docIdx, termFreq, -1)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기
            lexiconIdx = list(queryPosting.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = queryPosting[indexTerm]
            postingData = (lexiconIdx, docIdx, termFreq, beforeIdx)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx
            
#     print(localPosting)
# print(globalDocument)

#         if term not in globalLexicon.keys():
#             lexiconIdx = len(globalLexicon) 0

In [13]:
queryPosting, globalDocument

({'오히려': 55545,
  '단체측': 1,
  '무선이어폰': 4158,
  '연구는': 3,
  '존재치도': 4,
  '않아': 54721,
  '국내': 51733,
  '참여': 47736,
  '연구자도': 8,
  '어리둥절': 9,
  '이엠에프사이언티스에서': 10,
  '일부': 55278,
  '언론': 52586,
  '보도를': 46078,
  '부정하며': 14,
  '직접': 50631,
  '보내온': 18923,
  '이메일': 41130,
  '내용이다': 26251,
  '이엠에프사이언티스트': 19,
  '관계자인': 20,
  '조엘': 21,
  '모스코위츠': 22,
  '미국': 56588,
  '버클리': 24,
  '캘리포니아대': 25,
  '가정사회건강연구소': 26,
  '소장은': 9693,
  '무선': 6864,
  '이어폰의': 29,
  '건강': 41897,
  '유해성에': 31,
  '대한': 56647,
  '부정했다': 33,
  '고재원': 34,
  '1212': 35,
  '18일': 55005,
  '오전': 55460,
  '전세계': 38093,
  '과학자들이': 3259,
  '애플': 14865,
  '에어팟과': 4163,
  '같은': 54209,
  '이어폰이': 3263,
  '암': 3448,
  '발생': 43133,
  '위험을': 44176,
  '키울': 3267,
  '수': 56570,
  '있다는': 56733,
  '호소문을': 3270,
  '국제연합': 51,
  '과': 56270,
  '세계보건기구': 32961,
  '에': 55185,
  '제출했다는': 55,
  '국내외': 22092,
  '언론의': 57,
  '보도가': 47902,
  '나오면서': 39516,
  '불안감이': 60,
  '확산되고': 49008,
  '있다': 56540,
  '하지만': 56769,
  '실제': 53478,
  '이': 56326,
  '

In [14]:
globalPosting

[(0, 0, 1, -1),
 (1, 0, 1, -1),
 (2, 0, 1, -1),
 (3, 0, 2, -1),
 (4, 0, 1, -1),
 (5, 0, 1, -1),
 (6, 0, 1, -1),
 (7, 0, 1, -1),
 (8, 0, 1, -1),
 (9, 0, 1, -1),
 (10, 0, 1, -1),
 (11, 0, 4, -1),
 (12, 0, 2, -1),
 (13, 0, 2, -1),
 (14, 0, 1, -1),
 (15, 0, 2, -1),
 (16, 0, 1, -1),
 (17, 0, 2, -1),
 (18, 0, 1, -1),
 (19, 0, 2, -1),
 (20, 0, 2, -1),
 (21, 0, 2, -1),
 (22, 0, 2, -1),
 (23, 0, 2, -1),
 (24, 0, 2, -1),
 (25, 0, 2, -1),
 (26, 0, 2, -1),
 (27, 0, 2, -1),
 (28, 0, 5, -1),
 (29, 0, 2, -1),
 (30, 0, 2, -1),
 (31, 0, 1, -1),
 (32, 0, 4, -1),
 (33, 0, 1, -1),
 (34, 0, 2, -1),
 (35, 0, 2, -1),
 (36, 0, 2, -1),
 (37, 0, 1, -1),
 (38, 0, 3, -1),
 (39, 0, 1, -1),
 (40, 0, 3, -1),
 (41, 0, 2, -1),
 (42, 0, 2, -1),
 (43, 0, 1, -1),
 (44, 0, 1, -1),
 (45, 0, 1, -1),
 (46, 0, 1, -1),
 (47, 0, 1, -1),
 (48, 0, 2, -1),
 (49, 0, 1, -1),
 (50, 0, 2, -1),
 (51, 0, 1, -1),
 (52, 0, 3, -1),
 (53, 0, 1, -1),
 (54, 0, 4, -1),
 (55, 0, 1, -1),
 (56, 0, 2, -1),
 (57, 0, 1, -1),
 (58, 0, 2, -1),
 (59, 0

In [63]:
# 확인용
for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm:단어: postingIdx:위치, ...
    # print(indexTerm)
    
    while True: # Posting Next:-1
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        # print('  DocName:{0} - TermFreq:{1} - Next:{2}'.format(globalDocument[postingData[1]], postingData[2], postingData[3]))
        postingIdx = postingData[3]

In [15]:
globalPosting[queryPosting['오히려']]

(0, 176, 1, 54063)

In [16]:
globalPosting[globalPosting[queryPosting['오히려']][3]] # 다음 주소가 "-1" 일때 까지 반복해서 찾음

(0, 170, 1, 50284)

## TF

In [17]:
from math import log10

def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0

def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1+log10(freq)
    else:
        return 0

def doubleNormalTF(K, freq, maxFreq): 
    return K + ((1-K) * (freq/maxFreq))

In [19]:
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothigIDF(N, df):
    return log10((N+1)/df)

def probabilityIDF(N, df):
    return log10((N-df+1)/df)

In [20]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx

In [21]:
globalPosting

[[0, 0, 0.14285714285714285, -1],
 [1, 0, 0.14285714285714285, -1],
 [2, 0, 0.14285714285714285, -1],
 [3, 0, 0.2857142857142857, -1],
 [4, 0, 0.14285714285714285, -1],
 [5, 0, 0.14285714285714285, -1],
 [6, 0, 0.14285714285714285, -1],
 [7, 0, 0.14285714285714285, -1],
 [8, 0, 0.14285714285714285, -1],
 [9, 0, 0.14285714285714285, -1],
 [10, 0, 0.14285714285714285, -1],
 [11, 0, 0.5714285714285714, -1],
 [12, 0, 0.2857142857142857, -1],
 [13, 0, 0.2857142857142857, -1],
 [14, 0, 0.14285714285714285, -1],
 [15, 0, 0.2857142857142857, -1],
 [16, 0, 0.14285714285714285, -1],
 [17, 0, 0.2857142857142857, -1],
 [18, 0, 0.14285714285714285, -1],
 [19, 0, 0.2857142857142857, -1],
 [20, 0, 0.2857142857142857, -1],
 [21, 0, 0.2857142857142857, -1],
 [22, 0, 0.2857142857142857, -1],
 [23, 0, 0.2857142857142857, -1],
 [24, 0, 0.2857142857142857, -1],
 [25, 0, 0.2857142857142857, -1],
 [26, 0, 0.2857142857142857, -1],
 [27, 0, 0.2857142857142857, -1],
 [28, 0, 0.7142857142857143, -1],
 [29, 0, 0.

In [ ]:
N = len(globalDocument)
globalLexiconIDF = dict()

for indexTerm, postingIdx in globalLexicon.items():
    df = 0
    oldPostingIdx = postingIdx
    
    while True:  
        if postingIdx == -1:
            break
        
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    idf = basicIDF(N, df)
    globalLexiconIDF[indexTerm] = idf
    postingIdx = oldPostingIdx
    
    # print('{0} / IDF-{1}'.format(indexTerm, idf))
    
    while True:
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        globalPosting[postingIdx][2] *= idf
        # print('    Documents:{0} / TF:{1} / TF-IDF:{2}'.format(globalDocument[postingData[1]], 
        #                                                        TF, globalPosting[postingIdx][2]))
        
        postingIdx = postingData[3]

In [67]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()

dictTerm = list()
dictPos = list()
dictNoun = list()
dictNgram = list()
th = 1

for sentence in sent_tokenize(NewsContent[0]):
    for token in word_tokenize(sentence):
        if len(token) > th:
            dictTerm.append(token)
            # 아래서부터는 list이기때문에 extend를 사용해야 함. 
            dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
            dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
            dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
            
# 빠른 속도 및 중복 데이터 처리를 위해 set사용
dictTerm = list(set(dictTerm))
dictPos = list(set(dictPos))
dictNoun = list(set(dictNoun))
dictNgram = list(set(dictNgram))